In [1]:
import pandas as pd
import numpy as np
import warnings
import pickle

warnings.filterwarnings("ignore")

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

# 1.0 Data retrival

In [2]:
df = pd.read_pickle('../output/df_understanding.pkl')

In [3]:
df.head(2)

,Precipitazione dalle 9 alle 9 ( mm ),Precipitazione dalle 0 alle 0 ( mm ),Temperatura media ( °C ),Temperatura massima ( °C ),Temperatura minima ( °C ),Umidita' media ( % ),Umidita' massima ( % ),Umidita' minima ( % ),Velocita' media ( m/s ),Raffica ( m/s ),Durata Calma ( min ),Settore Prevalente,Tempo Permanenza ( min ),Radiazione totale ( MJ/mq ),precipitazione_media,evento_estremo,anticipatore
Data,,,,,,,,,,,,,,,,,
1988-05-05,3.8,3.6,14.7,22.6,9.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.7,0,0.0
1988-05-06,0.0,0.2,17.5,27.0,8.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,0,0.0


# 2.0 Data preparation
## 2.1 Imabalanced class

In [4]:
df['anticipatore'].value_counts()

0.0    11928
1.0      181
Name: anticipatore, dtype: int64

Given the large imbalance between classes present, it is necessary to treat the problem second:

- anomaly detection, or
- binary classifier after opportune oversampling or undersampling

The use of anomaly detection, being an alternative close to the unsupervised world, I do not consider it for the moment. I start with the binary classifier. For a first test I try the undersampling of the majority class: in this way I avoid to create bias that could occur in the case of oversampling of the minority class (through the creation of data, perhaps, too many fake).

In [5]:
df_under = df[df['anticipatore']==0]
df_over = df[df['anticipatore']==1]
df_under = df_under.sample(frac=df_over.shape[0]/df_under.shape[0],random_state=0)
df = df_over.append(df_under).sample(frac=1)

## 2.2 Train, dev, test split or Cross Validation?

In [6]:
df.shape

(362, 17)

Dealing with a relatively small dataset: should I do the 3-part split or is it better to use cross-validation?

Let's start with cross validation. In order to avoid data-leakage, I need to build a pipeline that correctly applies the following steps:

1) Filling holes
2) Rescaling
3) Classifier

## 2.3 Column datatype

In [7]:
df['Settore Prevalente'] = df['Settore Prevalente'].astype(str)

# 3.0 Output

Let's pass to the modeling phase

In [9]:
df.to_pickle('../output/df_prep.pkl')